In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
os.chdir("..")

In [3]:
from recoexplainer.config import cfg
from recoexplainer.data_reader.data_reader import DataReader
from recoexplainer.models.emf_model import EMFModel

from recoexplainer.recommender import RankPredictionsRecommender

In [4]:
data = DataReader(**cfg.ml100k)
data.make_consecutive_ids_in_dataset()

In [5]:
emf = EMFModel(**cfg.model.emf)

In [6]:
emf.fit(data)

True

In [7]:
rec = RankPredictionsRecommender(data, emf)

In [8]:
rec = rec.recommend_all()

In [9]:
rec

,userId,itemId,rank
243,0.0,277.0,1.0
645,0.0,715.0,2.0
190,0.0,221.0,3.0
1477,0.0,1569.0,4.0
924,0.0,1009.0,5.0
...,...,...,...
287,942.0,301.0,6.0
648,942.0,666.0,7.0
513,942.0,529.0,8.0
500,942.0,516.0,9.0


In [10]:
from recoexplainer.explain import EMFExplainer

In [11]:
explanations = EMFExplainer(emf, rec, data)

In [12]:
res = explanations.explain_recommendations()

In [13]:
res

,userId,itemId,rank,explanations
243,0.0,277.0,1.0,{4: 1}
645,0.0,715.0,2.0,{2: 1}
190,0.0,221.0,3.0,"{2: 1, 5: 1}"
1477,0.0,1569.0,4.0,{}
924,0.0,1009.0,5.0,"{3: 1, 5: 1}"
...,...,...,...,...
287,942.0,301.0,6.0,{}
648,942.0,666.0,7.0,{}
513,942.0,529.0,8.0,{}
500,942.0,516.0,9.0,{}
